#Projeto 3: Recomendações de animes

Este conjunto de dados contém informações sobre os dados de preferência do usuário de 73.516 usuários em 12.294 anime. Cada usuário pode adicionar anime à sua lista completa e dar-lhe uma classificação e este conjunto de dados é uma compilação dessas classificações.

***Tabelas:***

**Anime.csv**

anime_id - ID único de myanimelist.net que identifica um anime.

nome - nome completo do anime.

gênero - lista separada por vírgulas de gêneros para este anime.

tipo - filme, TV, OVA, etc.

episódios - quantos episódios neste show. (1 se filme).

classificação - classificação média de 10 para este anime.

membros - número de membros da comunidade que estão no
"grupo" deste anime .


**Rating.csv**

user_id - ID de usuário gerado aleatoriamente não identificável.

anime_id - o anime que este usuário avaliou.

rating - avaliação de 10 que este usuário atribuiu (-1 se o usuário assistiu, mas não atribuiu uma avaliação).

##Importação das bibliotecas básicas e carregamento dos dados

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
animes = pd.read_csv('/content/drive/MyDrive/Projetos python/Projeto 3: Recomendação de Filmes/anime.csv')
user = pd.read_csv('/content/drive/MyDrive/Projetos python/Projeto 3: Recomendação de Filmes/rating.csv')

In [ ]:
animes

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [ ]:
user

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


## Preparação dos dados

In [ ]:
user

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [ ]:
#Removendo os valores negativos que representam os usuários que não avaliaram os animes assistidos
def removneg(valor):
    if valor == -1:
        return np.NAN
    return valor

user['rating'] = user['rating'].apply(removneg)
user

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN
...,...,...,...
7813732,73515,16512,7.0
7813733,73515,17187,9.0
7813734,73515,22145,10.0
7813735,73516,790,9.0


In [ ]:
#Retirando os animes com menos de 10 avaliações
animeID = user['anime_id'].value_counts()
user = user[user['anime_id'].isin(animeID[animeID > 10].index)]
user['anime_id'].value_counts()

1535     39340
11757    30583
16498    29584
1575     27718
226      27506
         ...  
33119       11
6670        11
7405        11
12061       11
22169       11
Name: anime_id, Length: 7658, dtype: int64

In [ ]:
#Retirando os usuários que tem menos de 8 avaliações
userID = user['user_id'].value_counts()
user = user[user['user_id'].isin(userID[userID > 8].index)]
user['user_id'].value_counts()

48766    7433
42635    3619
53698    2813
57620    2682
59643    2624
         ... 
2457        9
20883       9
36779       9
52734       9
42395       9
Name: user_id, Length: 62651, dtype: int64

In [ ]:
user

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN
...,...,...,...
7813730,73515,13659,8.0
7813731,73515,14345,7.0
7813732,73515,16512,7.0
7813733,73515,17187,9.0


In [ ]:
#Selecionando apenas os tipos de animes de séries de TV
animes = animes[animes['type'] == 'TV']
juntar = user.merge(animes, left_on= 'anime_id', right_on='anime_id')
juntar = juntar[['user_id', 'anime_id', 'rating_x']]
juntar

,user_id,anime_id,rating_x
0,1,20,NaN
1,3,20,8.0
2,5,20,6.0
3,6,20,NaN
4,21,20,8.0
...,...,...,...
5246118,54472,2362,7.0
5246119,57821,2362,5.0
5246120,58108,2362,7.0
5246121,58305,2362,5.0


Fazendo a transposição da tabela para que possamos aplicar o algoritmo de recomendação

In [ ]:
userPivot = pd.pivot_table(juntar, values='rating_x', index='user_id', columns='anime_id')
userPivot

anime_id,1,6,7,8,15,16,17,18,19,20,22,23,24,25,26,27,28,29,30,33,45,48,50,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,...,32309,32313,32360,32370,32379,32438,32454,32483,32491,32526,32542,32595,32601,32606,32608,32648,32664,32681,32682,32696,32697,32729,32792,32802,32828,32906,32935,32947,32961,32998,33021,33028,33037,33046,33113,33222,33241,33341,33394,33421
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,8.0,NaN,NaN,6.0,NaN,6.0,6.0,NaN,6.0,5.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,6.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,8.0,7.0,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Invertendo as posições para que em linha fique os animes e em coluna os usuários
df_user = userPivot.T

In [ ]:
df_user.head()

user_id,1,3,5,7,8,11,12,14,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,37,38,39,40,41,43,44,45,46,47,48,50,51,...,73461,73462,73463,73464,73466,73468,73469,73470,73471,73473,73474,73476,73478,73480,73481,73483,73484,73485,73486,73488,73490,73491,73493,73494,73495,73497,73498,73499,73500,73501,73502,73503,73504,73506,73507,73510,73511,73512,73513,73515
anime_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,9.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,10.0,NaN,NaN,10.0,10.0,...,NaN,8.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,8.0,NaN,10.0,9.0,NaN,NaN,NaN,10.0,NaN,NaN,8.0,NaN,NaN,NaN,10.0,NaN,NaN,9.0,NaN,NaN,NaN,9.0,10.0,NaN,9.0,NaN,NaN,NaN,9.0,10.0
6,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,9.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,7.0,NaN,10.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,9.0,9.0,NaN,9.0,NaN,NaN,NaN,NaN,10.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_user.fillna(0, inplace=True)

In [ ]:
df_user

user_id,1,3,5,7,8,11,12,14,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,37,38,39,40,41,43,44,45,46,47,48,50,51,...,73461,73462,73463,73464,73466,73468,73469,73470,73471,73473,73474,73476,73478,73480,73481,73483,73484,73485,73486,73488,73490,73491,73493,73494,73495,73497,73498,73499,73500,73501,73502,73503,73504,73506,73507,73510,73511,73512,73513,73515
anime_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,10.0,0.0,0.0,10.0,10.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,8.0,0.0,10.0,9.0,0.0,0.0,0.0,10.0,0.0,0.0,8.0,0.0,0.0,0.0,10.0,0.0,0.0,9.0,0.0,0.0,0.0,9.0,10.0,0.0,9.0,0.0,0.0,0.0,9.0,10.0
6,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,7.0,0.0,10.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,9.0,9.0,0.0,9.0,0.0,0.0,0.0,0.0,10.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33341,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##Aplicação do Modelo de Recomendação

###Modelo de Fatoração de Matrizes


In [ ]:
# Importando a biblioteca
from sklearn.decomposition import TruncatedSVD

In [ ]:
#O Truncated SVD vai reduzir dimensão da matriz no número de componentes solicitado (10)
SVD_model = TruncatedSVD(n_components=10)
df_user_matrix = SVD_model.fit_transform(df_user)
df_user_matrix.shape

(2606, 10)

In [ ]:
#Gerando a matriz de correlação
correlation_matrix = np.corrcoef(df_user_matrix)
correlation_matrix.shape

(2606, 2606)

In [ ]:
#Selecionando o anime de entrada para gerar as recomendações
escolha = input(str('Nome do anime: '))

Nome do anime: Fullmetal Alchemist: Brotherhood


In [ ]:
#Pesquisando o nome do anime selecionado e retornando o ID do mesmo
anime_escolhe = animes[animes['name'] == escolha]
i = anime_escolhe['anime_id'].iat[0]

#identifando o index do ID do anime selecionado no banco de dados
product_names = list(df_user.index)
product_ID = product_names.index(i)

#Gerando as correlações para o anime escolhido
correlation_product_ID = correlation_matrix[product_ID]

In [ ]:
#Selecionado a recomendação utilizando uma correlação acima de 0.65
Recommend = list(df_user.index[correlation_product_ID > 0.65])

# Removendo o anime escolhido
Recommend.remove(i) 

#Gerando as top 10 animes mais recomendados
Recomendação = Recommend[0:10]
Recomendação

[1, 6, 19, 20, 25, 26, 30, 33, 67, 121]

In [ ]:
#Apresentando os 10 animes mais recomendados
anime = animes.query(f'anime_id == {Recomendação}')
anime['name']

22                     Cowboy Bebop
38                          Monster
151                         Berserk
200             Fullmetal Alchemist
211         Neon Genesis Evangelion
214                          Trigun
841                          Naruto
906                      Texhnolyze
1102    Basilisk: Kouga Ninpou Chou
1432                      Sunabouzu
Name: name, dtype: object